# Práctica 3 - Aplicación de Eurovisión. Fecha de Entrega: 5 de Mayo (Entrega temprana) / 18 de Mayo (Entrega tardía)

**NOTA**: esta práctica tiene dos fechas de entrega: el 5 y el 18 de mayo. Si entregáis la práctica **antes** del 5 de mayo (inclusive), tendréis la corrección antes del examen. Si estáis muy apurados con otras entregas/asignaturas, podéis entregar hasta el día 18 de Mayo y que os cuente para la convocatoria ordinaria. 

Haced una **primera lectura** superficial para entender lo que se os pide, y luego id desarrollando poco a poco la aplicación. Acostumbraos a ir probando vuestro código de forma continua, de manera que podáis delimitar los posibles problemas que vayan surgiendo. En la práctica, el testing (por ejemplo, _tests unitarios_) y la refactorización de código son claves para hacer que este tipo de proyectos sean escalables.

## Enunciado

¡Nuestra aplicación de *NBA Legends* fue un exitazo! La noticia ha corrido como la pólvora, y nos han pedido que desarrollemos una nueva aplicación. Esta vez, se trata de un proyecto aún más ambicioso: la UE, la Comunidad de Madrid y la UCM se han aliado para organizar el mayor concurso de trivia de la historia de Eurovisión en España. Juntando sus siglas, han formado la sociedad **EUCM**, encargada de supervisar el concurso **EUCMVisión**. Y nosotros nos encargaremos de desarrollar una aplicación web que permita a los futuros concursantes acceder a la información de todas las galas de Eurovisión para así poder prepararse. Y no solo eso, sino que también vamos a crear un generador de _quiz_ aleatorios, que permita a estos usuarios poner a prueba todos sus conocimientos.

## Modelos de Datos

En nuestra aplicación, disponemos de dos colecciones. Una de ellas, `festivales`, nos la proporcionan desde la **Unión Europea de Radiofusión** (**UER**) en el archivo `festivales.json`. Esta colección contiene toda la información sobre las distintas ediciones del festival que se han venido celebrando desde su iniciación. El siguiente código muestra un esquema de datos de ejemplo para el concurso de 1957:

```sh
{
  _id: ObjectId('67f2ab6b85e7d99fb0663051'),
  anyo: 1957,
  ciudad: 'Frankfurt',
  pais: 'Alemania',
  fecha: ISODate('1957-03-03T00:00:00.000Z'),
  concursantes: [
    {
      id_pais: 'nl',
      pais: 'Países Bajos',
      artista: 'Corry Brokken',
      cancion: 'Net als toen',
      resultado: 1,
      puntuacion: 31,
      url_youtube: 'https://www.youtube.com/embed/etHrsNTsJqM'
    },
    ...
    ]
}
```

Por otro lado, nosotros mismos vamos a mantener una colección de `quizzes`, en la que almacenaremos todos los _quizzes_ que se quieran almacenar. En nuestra aplicación, no vamos a considerar ningún usuario, así que no hay necesidad ni de proteger las rutas ni de guardar ninguna información extra. Por cada _quiz_, almacenaremos la lista de preguntas, respuestas y puntuación, así como un nombre (`_id`) único y la fecha de creación. Un documento de ejemplo es el siguiente:

```sh
{
_id: 'BV',
preguntas: [
  {
    correcta: 0,
    pregunta: "¿De que país es el intérprete de la canción 'Zero Gravity'?",
    puntuacion: 1,
    respuestas: [ 'Australia', 'Australia', 'Albania', 'Alemania' ],
    tipo: 'pregunta'
  }
],
creacion: ISODate('2025-04-06T22:59:20.494Z')
}
```


## Enrutado

Al igual que en la práctica anterior, el enrutado se encuentra descrito en el archivo `rutas.py`. Todas las rutas y sus métodos HTTP asociados están definidos, así que solo hay que seguir las indicaciones de los comentarios para implementar esos métodos. En los _templates_ se indica qué variables se deben suministrar y en qué formato.

## Restricciones de Diseño y Planteamiento de la Práctica

En esta práctica, se os proporcionan todos los templates ya generados, así como parte del enrutado. Esto se debe a que, en este caso, vamos a integrar ciertas partes de la aplicación con _JavaScript_. Esto no implica que vosotros vayáis a programar esta parte, pero es importante que veáis cómo se comunican el _frontend_ y el _backend_, y lo directa que es la integración de `MongoDB` con este lenguaje de programación.

Lo más importante es que todas las funcionalidades estén asociadas a las rutas pertinentes, y que el acceso al modelo de datos sea **eficiente**. Esto implica cargar la información imprescindible en cada caso, y hacerlo utilizando el menor número de consultas posibles (¡sin saturar la memoria en la ejecución!). En este caso, esto implica que a la hora de cargar la información, deberéis controlar qué información se carga utilizando proyecciones. En muchos casos, deberéis recurrir a agregaciones.

Una de las mayores diferencias con la práctica anterior es que en este caso no tenemos paginaciones de forma nativa, como en el caso de `SQLAlchemy`; sino que tenemos que implementarlas nosotros mismos. Por ello, he incluido una implementación de una paginación para `MongoDB` (en el módulo `render_utils`), así como el procesamiento de los argumentos de las peticiones HTTP. En vuestro caso, lo único que tenéis que controlar es cómo cargar la información de la página correspondiente, teniendo en cuenta el número de elementos por página (ya fijado). Además, debéis calcular el número total de documentos para determinar cuántas páginas hay en total.

## Paquete `trivia`

En la carpeta `app`, encontraréis una carpeta nueva: `trivia`. Esta carpeta contiene los métodos necesarios para generar preguntas aleatorias de acuerdo a 7 esquemas de preguntas: 4 de preguntas directas y 3 utilizando vídeos de **Youtube**. La idea es que en el módulo `__init__`, únicamente se exponen aquellos métodos que queremos que se invoquen de manera externa (desde `rutas.py`), y que el resto de módulos implementen la funcionalidad deseada. Esto ayuda a desacoplar el código y mejorar la reutilización del mismo. Cada vez que implementéis uno de estos 7 esquemas, tenéis que añadir la clase correspondiente a la lista `_preguntas_posibles`, de tal manera que el método `generar_n_preguntas_aleatoriamente` pueda seleccionarlas.

Por otro lado, se ha creado la clase `OperacionesEurovision` en el módulo `operaciones_coleccion`. Esta clase sirve como un modelo de la colección `festivales`, que nos permite abstraer la gestión de la colección del uso de la misma. Esta clase contiene ya implementados numerosos métodos que os permitirán generar información de manera aleatoria: años, países, participaciones... Contiene otros dos métodos adicionales para realizar consultas y agregaciones sobre la colección. Está **PROHIBIDO** acceder directamente a `self._coleccion`: tenéis que utilizar los métodos `consulta` y `agregacion` para comunicaros con la colección. A los atributos `self.anyos` y `self.paises` sí que podéis acceder de forma externa (se explica su uso en la plantilla de la práctica).

Los otros dos módulos, `preguntas.py` y `videos.py`, separan el manejo de los distintos esquemas de preguntas. Cada uno de ellos contiene una clase abstracta: `Trivia` y `TriviaVideo`, respectivamente. Estas clases especifican qué propiedades se espera de cada pregunta (con método `getters()`), de tal manera que podemos renderizar la información de una pregunta de trivia utilizando el método `to_dict`. Todos estos objetos deben recibir un único objeto en su constructor: `OperacionesEurovision`. En el constructor es donde se genera la información aleatoriamente, que posteriormente se utilizará para instanciar las propiedades abstractas.

## ¿Cómo abordar la práctica?

A diferencia de la práctica anterior, en este caso, casi todas las funcionalidades del sistema son independientes. Os recomiendo que empecéis primero mostrando la información que ya tenéis en la base de datos. La función de vista más sencilla es `mostrar_festival`, seguida de `mostrar_ediciones`. 

Una vez os hayáis acostumbrado a las consultas en `MongoDB` y al renderizado de _templates_, podéis proseguir con`mostrar_actuaciones_pais`. A partir de este momento, el resto de funcionalidades están vinculadas a las de un _quiz_. El siguiente paso es probar la parte de jugar (`jugar_quiz`) con toda la parte del paquete `trivia`. Esta parte a veces puede dar problemas con ciertos vídeos que no se cargan bien. Más adelante podréis probar con los quizzes personalizados y con las ediciones más recientes de Eurovisión, que suelen funcionar sin problemas. Una vez lo tengáis, probad a generar _quizzes_ personalizados (`generar_quiz`) y guardar su información en la colección correspondiente (`guardar_concurso`). A partir de este momento, podéis probar a mostrar esos concursos (`mostrar_quizzes`) y a cargarlos (`jugar_quiz_personalizado`).

## Índices

La última parte de la práctica consiste en revisar la eficiencia de las consultas que realizáis, y aprovechéis para contestar a la siguiente cuestión:

1. ¿Qué índices creéis que deberíais declarar en vuestras colecciones para mejorar las consultas?

Escribid vuestra respuesta (un par de párrafos en `.txt` o **Word**) en un archivo `indices.txt`.

## Entrega de la Práctica

Subid un archivo `*.zip` con el código de vuestra aplicación. Incluid un archivo `alumnos.txt` con el nombre y apellidos de los participantes de la práctica y el grupo, así como el archivo `indices.txt` con vuestro razonamiento del apartado anterior.